In [1]:
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd
import numpy as np

In [2]:
def getResults():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432' password='MKeeJn9zEqs2T9wm4V3UX'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT r.resultid, r.resultdatetime, rv.valueid, rv.datavalue, rv.valuedatetime
                       FROM results r 
                       INNER JOIN measurement_result_values rv ON r.resultid=rv.resultid
                       INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid 
                       INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                       WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'""")
        rows = cur.fetchall()
        
        df = pd.DataFrame(columns = ['resultid', 'resultdatetime', 'valueid', 'datavalue', 'valuedatetime']) 
        
        for row in rows:
            df = df.append({'resultid': row[0], 'resultdatetime': row[1], 'valueid': row[2], 
                            'datavalue': row[3], 'valuedatetime': row[4]}, 
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [3]:
dfdb = getResults()
dfdb.head()

,resultid,resultdatetime,valueid,datavalue,valuedatetime
0,10191,2018-10-22 00:00:00-04:00,202,945.413,2018-10-22 00:00:00-04:00
1,10194,2018-10-22 00:00:00-04:00,197,25.326,2018-10-22 00:00:00-04:00
2,10195,2018-10-22 00:00:00-04:00,198,54.794,2018-10-22 00:00:00-04:00
3,10196,2018-10-22 00:00:00-04:00,195,8.455,2018-10-22 00:00:00-04:00
4,10197,2018-10-22 00:00:00-04:00,201,357.060,2018-10-22 00:00:00-04:00


In [4]:
len(np.unique(np.array(dfdb['valueid'])))

381

In [5]:
len(dfdb['valueid'])

381

In [6]:
len(np.unique(np.array(dfdb['resultid'])))

381

In [7]:
len(dfdb['resultid'])

381

In [8]:
def getActions():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432' password='MKeeJn9zEqs2T9wm4V3UX'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT r.resultid, r.resultdatetime, rv.valueid, rv.datavalue, rv.valuedatetime,
                                       a.actionid, a.begin_datetime, a.end_datetime, a.methodid
                       FROM results r 
                       INNER JOIN measurement_result_values rv ON r.resultid=rv.resultid
                       INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid
                       INNER JOIN actions a ON fa.actionid=a.actionid
                       INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                       WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'""")
        rows = cur.fetchall()
        
        df = pd.DataFrame(columns = ['resultid', 'resultdatetime', 'valueid', 'datavalue', 'valuedatetime',
                                     'actionid', 'begin_datetime', 'end_datetime', 'methodid']) 
        
        for row in rows:
            df = df.append({'resultid': row[0], 'resultdatetime': row[1], 'valueid': row[2], 
                            'datavalue': row[3], 'valuedatetime': row[4], 'actionid': row[5], 
                            'begin_datetime': row[6], 'end_datetime': row[7], 'methodid': row[8]}, 
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [9]:
dfdb = getActions()
dfdb.head()

,resultid,resultdatetime,valueid,datavalue,valuedatetime,actionid,begin_datetime,end_datetime,methodid
0,10191,2018-10-22 00:00:00-04:00,202,945.413,2018-10-22 00:00:00-04:00,118,2018-10-22 00:00:00-04:00,2018-10-22 00:00:00-04:00,10004
1,10194,2018-10-22 00:00:00-04:00,197,25.326,2018-10-22 00:00:00-04:00,118,2018-10-22 00:00:00-04:00,2018-10-22 00:00:00-04:00,10004
2,10195,2018-10-22 00:00:00-04:00,198,54.794,2018-10-22 00:00:00-04:00,118,2018-10-22 00:00:00-04:00,2018-10-22 00:00:00-04:00,10004
3,10196,2018-10-22 00:00:00-04:00,195,8.455,2018-10-22 00:00:00-04:00,118,2018-10-22 00:00:00-04:00,2018-10-22 00:00:00-04:00,10004
4,10197,2018-10-22 00:00:00-04:00,201,357.060,2018-10-22 00:00:00-04:00,118,2018-10-22 00:00:00-04:00,2018-10-22 00:00:00-04:00,10004


In [10]:
len(np.unique(np.array(dfdb['actionid'])))

30

In [11]:
len(dfdb['actionid'])

381

In [12]:
len(np.unique(np.array(dfdb['end_datetime'])))

1

In [13]:
def getFeatures():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432' password='MKeeJn9zEqs2T9wm4V3UX'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT r.resultid, r.resultdatetime, rv.valueid, rv.datavalue, rv.valuedatetime,
                                       a.actionid, a.begin_datetime, a.end_datetime, a.methodid,
                                        sf.samplingfeatureid, ST_X(ST_AsText(sf.featuregeometry))
                       FROM results r 
                       INNER JOIN measurement_result_values rv ON r.resultid=rv.resultid
                       INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid
                       INNER JOIN actions a ON fa.actionid=a.actionid
                       INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                       WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'""")
        rows = cur.fetchall()
        
        df = pd.DataFrame(columns = ['resultid', 'resultdatetime', 'valueid', 'datavalue', 'valuedatetime',
                                     'actionid', 'begin_datetime', 'end_datetime', 'methodid',
                                      'samplingfeatureid', 'lon']) 
        
        for row in rows:
            df = df.append({'resultid': row[0], 'resultdatetime': row[1], 'valueid': row[2], 
                            'datavalue': row[3], 'valuedatetime': row[4], 'actionid': row[5], 
                            'begin_datetime': row[6], 'end_datetime': row[7], 'methodid': row[8],
                            'samplingfeatureid': row[9], 'lon': row[10]},
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [14]:
dfdb = getFeatures()
dfdb.head()

,resultid,resultdatetime,valueid,datavalue,valuedatetime,actionid,begin_datetime,end_datetime,methodid,samplingfeatureid,lon
0,10191,2018-10-22 00:00:00-04:00,202,945.413,2018-10-22 00:00:00-04:00,118,2018-10-22 00:00:00-04:00,2018-10-22 00:00:00-04:00,10004,18847,None
1,10194,2018-10-22 00:00:00-04:00,197,25.326,2018-10-22 00:00:00-04:00,118,2018-10-22 00:00:00-04:00,2018-10-22 00:00:00-04:00,10004,18847,None
2,10195,2018-10-22 00:00:00-04:00,198,54.794,2018-10-22 00:00:00-04:00,118,2018-10-22 00:00:00-04:00,2018-10-22 00:00:00-04:00,10004,18847,None
3,10196,2018-10-22 00:00:00-04:00,195,8.455,2018-10-22 00:00:00-04:00,118,2018-10-22 00:00:00-04:00,2018-10-22 00:00:00-04:00,10004,18847,None
4,10197,2018-10-22 00:00:00-04:00,201,357.060,2018-10-22 00:00:00-04:00,118,2018-10-22 00:00:00-04:00,2018-10-22 00:00:00-04:00,10004,18847,None


In [15]:
np.array(dfdb['lon'])

array([None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, No

In [16]:
len(np.unique(np.array(dfdb['samplingfeatureid'])))

30